In [ ]:
# 모듈, 레이어 및 모델
# 모델은 추상적으로 텐서에서 무언가를 계산하는 함수(정방향 전달)나 훈련에 대한 응답으로 업데이트할 수 있는 일부 변수


# TensorFlow에서 모델 및 레이어 정의
# 대부분의 모델은 레이어(재사용할 수 있고 훈련 가능한 변수를 가진, 알려진 수학적 구조의 함수)로 구성
# TensorFlow에서 Keras 또는 Sonnet과 같은 레이어 및 모델의 상위 수준 구현 대부분은 같은 기본 클래스인 tf.Module를 기반으로 구축
import tensorflow as tf
from datetime import datetime

# %load_ext tensorboard

# class SimpleModule(tf.Module):
#     def __init__(self, name=None):
#         super().__init__(name=name)
#         self.a_variable = tf.Variable(5.0, name='train_me')
#         self.non_trainable_variable = tf.Variable(5.0, trainable=False, name='do_not_train_me')

#     def __call__(self, x):
#         return self.a_variable * x + self.non_trainable_variable

# simple_module = SimpleModule(name='simple')
# simple_module(tf.constant(5.0))

# 모듈과 더 나아가 레이어는 '객체'에 대한 딥러닝 용어이며, 내부 상태와 해당 상태를 사용하는 메서드가 존재
# 미세 조정 중 레이어 및 변수 고정을 포함하여 어떤 이유로든 변수의 훈련 가능성을 설정 및 해제 가능
# tf.Module을 하위 클래스화함으로써 이 객체의 속성에 할당된 tf.Variable 또는 tf.Module 인스턴스가 자동으로 수집
# 이를 통해 변수를 저장 및 로드할 수 있으며, tf.Module 모음을 만들 수 있음
# print("Trainable variables:", simple_module.trainable_variables)
# print("All variables:", simple_module.variables)

# 밀집(선형) 레이어
# class Dense(tf.Module):
#     def __init__(self, in_features, out_features, name=None):
#         super().__init__(name=name)
#         self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
#         self.b = tf.Variable(tf.zeros([out_features]), name='b')

#     def __call__(self, x):
#         y = tf.matmul(x, self.w) + self.b
#         return tf.nn.relu(y)

# 2개의 레이어 인스턴스를 만들고 적용하는 전체 모델
# class SequentialModule(tf.Module):
#     def __init__(self, name=None):
#         super().__init__(name=name)
#         self.dense_1 = Dense(in_features=3, out_features=3)
#         self.dense_2 = Dense(in_features=3, out_features=2)

#     def __call__(self, x):
#         x = self.dense_1(x)
#         return self.dense_2(x)

# my_model = SequentialModule(name='the_model')
# print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))
# print("Submodules:", my_model.submodules)

# for var in my_model.variables:
#     print(var, "\n")

## 변수 생성 연기
# 특정 입력 형상으로 모듈이 처음 호출될 때까지 변수 생성을 연기하면 입력 크기를 미리 지정할 필요가 없음
# class FlexibleDenseModule(tf.Module):
#     def __init__(self, out_features, name=None):
#         super().__init__(name=name)
#         self.is_built = False
#         self.out_features = out_features

#     def __call__(self, x):
#         # Create variables on first call
#         if not self.is_built:
#             self.w = tf.Variable(tf.random.normal([x.shape[-1], self.out_features]), name='w')
#             self.b = tf.Variable(tf.zeros([self.out_features]), name='b')
#             self.is_built = True

#         y = tf.matmul(x, self.w) + self.b
#         return tf.nn.relu(y)

# class MySequentialModule(tf.Module):
#     def __init__(self, name=None):
#         super().__init__(name=name)
#         self.dense_1 = FlexibleDenseModule(out_features=3)
#         self.dense_2 = FlexibleDenseModule(out_features=2)

#     def __call__(self, x):
#         x = self.dense_1(x)
#         return self.dense_2(x)

# my_model = MySequentialModule(name='the_model')
# print("Model results:", my_model(tf.constant([[2.0, 2.0, 2.0]])))


# 가중치 저장
# tf.Module은 checkpoint와 SavedModel로 모두 저장 가능
# 체크포인트는 가중치(모듈 및 하위 모듈 내부의 변수 세트 값)
# chkp_path = "my_checkpoint"
# checkpoint = tf.train.Checkpoint(model=my_model)
# checkpoint.write(chkp_path)

# 체크포인트는 데이터 자체와 메타데이터용 인덱스 파일로 구성
# 인덱스 파일은 실제로 저장된 항목과 체크포인트 번호를 추적하는 반면 체크포인트 데이터에는 변수 값과 해당 속성 조회 경로가 포함
# !ls my_checkpoint*

# tf.train.list_variables(chkp_path)

# 분산 훈련 중에 변수 모음이 샤딩될 수 있으므로 번호가 매겨짐(예: '00000-of-00001')
# 이 경우에는 샤드가 하나만 존재
# new_model = MySequentialModule()
# new_checkpoint = tf.train.Checkpoint(model=new_model)
# new_checkpoint.restore("my_checkpoint")

# Should be the same result as above
# new_model(tf.constant([[2.0, 2.0, 2.0]]))


# 함수 저장
# class Dense(tf.Module):
#     def __init__(self, in_features, out_features, name=None):
#         super().__init__(name=name)
#         self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
#         self.b = tf.Variable(tf.zeros([out_features]), name='b')

#     def __call__(self, x):
#         y = tf.matmul(x, self.w) + self.b
#         return tf.nn.relu(y)

# class MySequentialModule(tf.Module):
#     def __init__(self, name=None):
#         super().__init__(name=name)
#         self.dense_1 = Dense(in_features=3, out_features=3)
#         self.dense_2 = Dense(in_features=3, out_features=2)

#     @tf.function
#     def __call__(self, x):
#         x = self.dense_1(x)
#         return self.dense_2(x)

# A model with a graph
# my_model = MySequentialModule(name='the_model')

# print(my_model([[2.0, 2.0, 2.0]]))
# print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))

# TensorBoard 요약 내에서 그래프를 추적해 그래프 시각화
# Set up logging
# stamp = datetime.now().strftime("%Y%m%d-%H%M%S")
# logdir = "logs/func/%s" % stamp
# writer = tf.summary.create_file_writer(logdir)

# Create a new model to get a fresh trace otherwise the summary will not see the graph
# new_model = MySequentialModule()

# Bracket the function call with tf.summary.trace_on() an tf.summary.trace_export()
# tf.summary.trace_on(graph=True)
# tf.profiler.experimental.start(logdir)

# Call only one tf.function when tracing
# z = print(new_model(tf.constant([[2.0, 2.0, 2.0]])))
# with writer.as_default():
#     tf.summary.trace_export(
#         name='my_func_trace',
#         step=0,
#         profiler_outdir=logdir)

# docs_infra: no execute
# %tensorboard --logdir logs/func

## SavedModel 생성
# 완전히 훈련된 모델을 공유하는 권장 방법은 SavedModel을 사용하는 방법
# SavedModel에는 함수 모음과 가중치 모음이 모두 포함
# tf.saved_model.save(my_model, 'the_saved_model')

# saved_model.pb 파일은 함수형 tf.Graph를 설명하는 프로토콜 버퍼
# Inspect the SavedModel in the directory
# !ls -l the_saved_model

# The variables/ directory contains a checkpoint of the variables
# !ls -l the_saved_model/variables

# new_model = tf.saved_model.load('the_saved_model')

# 저장된 모델을 로드하여 생성된 new_model은 클래스 지식이 없는 내부 TensorFlow 사용자 객체
# isinstance(new_model, SequentialModule)

# 저장된 모델을 로드하여 생성된 모델은 이미 정의된 입력 서명에서 동작, 서명 추가 불가능
# print(my_model([[2.0, 2.0, 2.0]]))
# print(my_model([[[2.0, 2.0, 2.0], [2.0, 2.0, 2.0]]]))


# Keras 모델 및 레이어
## Keras 레이어
# tf.keras.layers.Layer는 모든 Keras 레이어의 기본 클래스이며, tf.Module을 상속
# 부모를 교체한 다음, __call__을 call로 변경하여 모듈을 Keras 레이어로 변환 가능
# class MyDense(tf.keras.layers.Layer):
#     # Adding **kwargs to support base Keras layer arguments
#     def __init__(self, in_features, out_features, **kwargs):
#         super().__init__(**kwargs)
#         self.w = tf.Variable(tf.random.normal([in_features, out_features]), name='w')
#         self.b = tf.Variable(tf.zeros([out_features]), name='b')

#     def call(self, x):
#         y = tf.matmul(x, self.w) + self.b
#         return tf.nn.relu(y)

# simple_layer = MyDense(name='simple', in_features=3, out_features=3)
# simple_layer([[2.0, 2.0, 2.0]])

## build 단계
# 입력 형상이 확실해질 때까지 변수를 생성하기 위해 기다리는 것이 많은 경우 편리
# Keras 레이어에는 레이어를 정의하는 방법에 더 많은 유연성을 제공하는 추가 수명 주기 단계가 있고, build 함수에서 정의
# build는 정확히 한 번만 호출되며 입력 형상으로 호출됨
class FlexibleDense(tf.keras.layers.Layer):
    # Note the added '**kwargs', as Keras supports many arguments
    def __init__(self, out_features, **kwargs):
        super().__init__(**kwargs)
        self.out_features = out_features

    # Create the state of the layer(weights)
    def build(self, input_shape):
        self.w = tf.Variable(tf.random.normal([input_shape[-1], self.out_features]), name='w')
        self.b = tf.Variable(tf.zeros([self.out_features]), name='b')

    # Defines the computation from inputs to outputs
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

# flexible_dense = FlexibleDense(out_features=3)

# 이 시점에는 모델이 빌드되지 않았으므로 변수가 없음
# flexible_dense.variables

# print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0], [3.0, 3.0, 3.0]])))

# flexible_dense.variables

# build는 한 번만 호출되므로 입력 형상이 레이어의 변수와 호환되지 않으면 입력이 거부됨
# try:
#     print("Model results:", flexible_dense(tf.constant([[2.0, 2.0, 2.0, 2.0]])))
# except tf.errors.InvalidArgumentError as e:
#     print("Failed:", e)

# Keras 레이어에는 다음과 같은 더 많은 추가 기능이 존재
# 1. 선택적 손실
# 2. 메트릭 지원
# 3. 훈련 및 추론 사용을 구분하기 위한 선택적 tracing 인수에 대한 기본 지원
# 4. Python에서 모델 복제를 허용하도록 구성을 정확하게 저장할 수 있는 get_config 및 from_config 메서드

## Keras 모델
# 모델을 중첩된 Keras 레이어로 정의 가능하지만, Keras는 tf.keras.Model이라는 완전한 기능을 갖춘 모델 클래스도 제공
# tf.keras.layers.Layer에서 상속되므로 Keras 모델은 Keras 레이어와 마찬가지로 사용, 중첩 및 저장 가능
# Keras 모델에는 쉽게 훈련, 평가, 로드 및 저장하고, 심지어 여러 머신에서 훈련할 수 있는 추가 기능 존재
class MySequentialModel(tf.keras.Model):
    def __init__(self, name=None, **kwargs):
        super().__init__(**kwargs)
        self.dense_1 = FlexibleDense(out_features=3)
        self.dense_2 = FlexibleDense(out_features=2)

    def call(self, x):
        x = self.dense_1(x)
        return self.dense_2(x)

my_sequential_model = MySequentialModel(name='the_model')

# print("Model results:", my_sequential_model(tf.constant([[2.0, 2.0, 2.0]])))

# my_sequential_model.variables
# my_sequential_model.submodules

# tf.keras.Model을 재정의하는 것은 TensorFlow 모델을 빌드하는 Python다운 접근 방식
# 다른 프레임워크에서 마이그레이션할 경우 매우 간단할 가능성
# 기존 레이어와 입력을 간단하게 조합한 모델을 구성하는 경우,
# 모델 재구성 및 아키텍처와 관련된 추가 기능과 함께 제공되는 함수형 API를 사용해 시간과 공간 절약 가능

# 험수형 API가 있는 동일한 모델
# 가장 큰 차이점은 입력 형상이 함수형 구성 프로세스의 일부로 미리 지정
# 이 경우, input_shape 인수를 완전히 지정할 필요가 없으며, 일부 차원은 None으로 남겨둘 수 있음
# inputs = tf.keras.Input(shape=[3,])

# x = FlexibleDense(3)(inputs)
# x = FlexibleDense(2)(x)

# my_functional_model = tf.keras.Model(inputs=inputs, outputs=x)
# my_functional_model.summary()

# my_functional_model(tf.constant([[2.0, 2.0, 2.0]]))


# Keras 모델 저장
# Keras 모델에서는 체크포인트를 사용할 수 있으며, tf.Module과 같게 보임
# Keras 모델은 모듈 tf.saved_models.save()로 저장할 수도 있으나 Keras 모델에는 편리한 메서드와 기타 기능이 존재
my_sequential_model.save("exname_of_file")
reconstructed_model = tf.keras.models.load_model("exname_of_file")

# Keras SavedModel는 또한 메트릭, 손실 및 옵티마이저 상태를 저장
# 재구성된 모델을 사용할 수 있으며, 같은 데이터로 호출될 때 같은 결과 생성
reconstructed_model(tf.constant([[2.0, 2.0, 2.0]]))